In [ ]:
# !pip install numpy==1.26.3 && pip install pandas==2.2.0 && pip install torch==2.1.2 && pip install transformers==4.37.0 && pip install datasets==2.1.0 && pip install evaluate==0.4.1 && pip install seqeval==1.2.2 && pip install peft==0.8.2

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, TokenClassificationPipeline
from transformers.pipelines import AggregationStrategy
from peft import (
    PeftModel,
    get_peft_model,
    LoraConfig,
    TaskType
)

# loading dataset and tokenizer

In [ ]:

WINDOW_LENGTH = 512
STRIDE = 256



In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [ ]:
dataset = load_dataset("HaniehPoostchi/persian_ner")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset persian_ner downloaded and prepared to /root/.cache/huggingface/datasets/HaniehPoostchi___persian_ner/fold1/1.1.0/c3ef2ac869a37d2abd8de5109e1a8ce418d360465093dfaf050f8cb5b2e46cc0. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 5121
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2560
    })
})

In [ ]:
dataset['train'][0]

{'tokens': ['به',
  'عنوان',
  'مثال',
  'وقتی',
  'نشریات',
  'مدافع',
  'اصول',
  'و',
  'ارزشها',
  'و',
  'منادی',
  'انقلاب',
  'و',
  'اسلام',
  'در',
  'بالاترین',
  'درجه',
  '،',
  'اولین',
  'و',
  'درشت\u200cترین',
  'تیتر',
  'نشریه',
  'خود',
  'را',
  'در',
  'صدر',
  'صفحه',
  'نخستین',
  '،',
  'به',
  'تکذیب',
  'اظهارات',
  'و',
  'نظریات',
  'مشاور',
  'رئیس\u200cجمهور',
  'با',
  'همین',
  'ترکیب',
  'عبارتی',
  'و',
  'البته',
  'از',
  'قول',
  'دیگران',
  'اختصاص',
  'می\u200cدهند',
  '،',
  'آیا',
  'در',
  'موارد',
  'مشابه',
  'نیز',
  'هر',
  'گاه',
  'خبر',
  'تکذیب',
  'متوجه',
  'و',
  'معطوف',
  'به',
  'شخصی',
  'باشد',
  'كه',
  'در',
  'زمره',
  'مشاوران',
  'يك',
  'مقام',
  'بلندمرتبه\u200cی',
  'دیگر',
  'است',
  '،',
  'خبر',
  'را',
  'عینا',
  'به',
  'همین',
  'درشتی',
  'و',
  'با',
  'همین',
  'ترکیب',
  'عبارتی',
  'در',
  'صدر',
  'صفحه',
  'نخست',
  'به',
  'چاپ',
  'می\u200cرساند',
  'و',
  'در',
  'آن',
  'مورد',
  'هم',
  'به',
  'جای',


In [ ]:
label_list = dataset["train"].features[f"ner_tags"].feature.names

In [ ]:
label_list

['O',
 'I-event',
 'I-fac',
 'I-loc',
 'I-org',
 'I-pers',
 'I-pro',
 'B-event',
 'B-fac',
 'B-loc',
 'B-org',
 'B-pers',
 'B-pro']

In [ ]:
# llm_name = "HooshvareLab/bert-base-parsbert-ner-uncased"
llm_name = "HooshvareLab/bert-base-parsbert-armanner-uncased"
tok_kwargs = {
    'model_max_length': WINDOW_LENGTH,
    'truncation': True,
    'padding': "max_length",
    'stride': STRIDE,
    'return_overflowing_tokens': True,
    'is_split_into_words': True
}


tokenizer = AutoTokenizer.from_pretrained(
    llm_name,
    **tok_kwargs
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/937 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

preprocessing labels

In [ ]:
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast
from datasets.arrow_dataset import Batch
# from datasets import Batch
from transformers.tokenization_utils_base import BatchEncoding

In [ ]:
# without label id
def tokenize_and_align_labels(
        instances: dict,
        llm_tok: PreTrainedTokenizer | PreTrainedTokenizerFast
        ) -> BatchEncoding:
    """Divides tokens of input data into sliding windows and aligns labels.

    Upon tokenizing the input data into the defined window lengths, the labels
    are rearranged to match the shape of the tokenized text. This is done by
    iterating over every sliding window that was just created and creating
    a separate list of labels for each sliding window. This function follows
    the logic of the `tokenize_and_align_labels` function from the huggingface
    tutorial (https://huggingface.co/docs/transformers/tasks/token_classification)
    and makes the said function compatible to the use case of passing large text
    data into an LLM through sliding windows.

    Args:
        instances: One or more row(s) of a huggingface dataset.
        llm_tok: Initialized LLM tokenizer.
    Returns:
        Tokenized instances with aligned labels.
    """
    tokenized_instances = llm_tok(
        instances["tokens"],
        truncation=True,
        padding="max_length",
        max_length=WINDOW_LENGTH,
        stride=STRIDE,
        return_overflowing_tokens=True,
        is_split_into_words=True
    )

    labels_input = instances["ner_tags"]
    labels = []
    n_windows = len(tokenized_instances['input_ids'])
    for window_i, instance_i in zip(
        range(n_windows),
        tokenized_instances['overflow_to_sample_mapping']
    ):
        window_labels = []
        word_indexes = tokenized_instances.word_ids(window_i)
        previous_word_idx = None

        for word_idx in word_indexes:  # Set the special tokens to -100.
            if word_idx is None:
                window_labels.append(-100)
            # Only label the first token of a given word.
            # Assign label to right sublist if input overflows.
            elif word_idx != previous_word_idx:
                window_labels.append(labels_input[instance_i][word_idx])
            else:
                window_labels.append(-100)
            previous_word_idx = word_idx

        labels.append(window_labels)

    tokenized_instances["labels"] = labels

    return tokenized_instances


In [ ]:
import os

import numpy as np
import torch
import evaluate
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)

label_list = dataset["train"].features[f"ner_tags"].feature.names

dataset_tokenized = dataset.map(
    tokenize_and_align_labels, fn_kwargs={"llm_tok": tokenizer}, batched=True,
    remove_columns=dataset["train"].column_names
    )

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(
        predictions=true_predictions,
        references=true_labels
    )
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

id2label = {i:label for i, label in enumerate(label_list)}
label2id = {label:i for i, label in id2label.items()}



  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
dataset_tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 5121
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 2560
    })
})

In [ ]:
label2id

{'O': 0,
 'I-event': 1,
 'I-fac': 2,
 'I-loc': 3,
 'I-org': 4,
 'I-pers': 5,
 'I-pro': 6,
 'B-event': 7,
 'B-fac': 8,
 'B-loc': 9,
 'B-org': 10,
 'B-pers': 11,
 'B-pro': 12}

# loading the model

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    llm_name,
    num_labels=13,
    id2label=id2label,
    label2id=label2id,

)

pytorch_model.bin:   0%|          | 0.00/651M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-armanner-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing 

# fine-tuning


In [ ]:
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType
)


tar_modules = ["key", "query", "value"]

peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="all",
    target_modules=tar_modules,
    modules_to_save=["classifier"],
)


lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

trainable params: 996,877 || all params: 163,155,482 || trainable%: 0.610998164315435


In [ ]:
lora_model.to(device)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/parsBert/model_training",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"  # disable w&b logging
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model('/content/drive/MyDrive/parsBert/lora_model_fintetuned')

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.201400,0.094573,0.658037,0.787150,0.716826,0.978933
2,0.030300,0.025163,0.914979,0.950081,0.932200,0.994812
3,0.013400,0.014804,0.952047,0.972729,0.962277,0.996797
4,0.007600,0.012332,0.958693,0.976196,0.967365,0.997138
5,0.005900,0.010708,0.964212,0.977583,0.970852,0.997443
6,0.004600,0.010141,0.968236,0.979200,0.973687,0.997565
7,0.005200,0.009935,0.969787,0.979200,0.974471,0.997540
8,0.004900,0.009605,0.969371,0.980125,0.974718,0.997625
9,0.003100,0.009425,0.969322,0.978507,0.973893,0.997601
10,0.003000,0.009361,0.968743,0.981280,0.974971,0.997650


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.201400,0.094573,0.658037,0.787150,0.716826,0.978933
2,0.030300,0.025163,0.914979,0.950081,0.932200,0.994812
3,0.013400,0.014804,0.952047,0.972729,0.962277,0.996797
4,0.007600,0.012332,0.958693,0.976196,0.967365,0.997138
5,0.005900,0.010708,0.964212,0.977583,0.970852,0.997443
6,0.004600,0.010141,0.968236,0.979200,0.973687,0.997565
7,0.005200,0.009935,0.969787,0.979200,0.974471,0.997540
8,0.004900,0.009605,0.969371,0.980125,0.974718,0.997625
9,0.003100,0.009425,0.969322,0.978507,0.973893,0.997601
10,0.003000,0.009361,0.968743,0.981280,0.974971,0.997650


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from transformers import AutoModelForTokenClassification
from peft import PeftModel


# instantiate base model
model = AutoModelForTokenClassification.from_pretrained(
    llm_name,
    num_labels=13,
    id2label=id2label,
    label2id=label2id,

)
# load LoRA model
inference_model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/parsBert/lora_model_fintetuned")
# merge and save model
merged_model = inference_model.merge_and_unload()
merged_model.save_pretrained("/content/drive/MyDrive/parsBert/full_finetuned_model")

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-armanner-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# inference

In [ ]:
#pipe = pipeline("token-classification", model="./saved_model_4", tokenizer=tokenizer, aggregation_strategy="simple")

from transformers import pipeline

# assert that we use fast version
assert tokenizer.is_fast

# instantiate pipeline
classifier_1 = pipeline(
    "token-classification",
    model="/content/drive/MyDrive/parsBert/full_finetuned_model",
    tokenizer=tokenizer,
    stride=STRIDE,  # additional parameter
    aggregation_strategy='max'
)

# standard token classification pipeline expects untokenized input
sentence = " ".join(dataset['test']['tokens'][0])
# retokenize sentence for subsequent print statements
tokenized_sentence = tokenizer(
    sentence,
    truncation=True,
    padding="max_length",
    max_length=WINDOW_LENGTH,
    stride=STRIDE,
    return_overflowing_tokens=True,
    is_split_into_words=False
)
# get text length (i.e. number of characters) of first window
text_window_1 = " ".join(
    [tokenizer.decode(tkn) for tkn in tokenized_sentence['input_ids'][0]]
)
n_words = len(dataset['test']['tokens'][0])

print(f"total number of words: {n_words}, total length of text: {len(sentence)}")
# returns: total number of words: 1350, total length of text: 6899
print(f"length of text in first window: {len(text_window_1)}")
# returns: length of text in first window: 2542

output = classifier_1(sentence)

last_token = output[-1]
print(f"last word tagged at text position: {last_token['start']}-{last_token['end']}")
# returns: last word tagged at text position: 6862-6869

total number of words: 240, total length of text: 1083
length of text in first window: 2926
last word tagged at text position: 1018-1025


In [ ]:
print(output)

[{'entity_group': 'loc', 'score': 0.9957581, 'word': 'ایران', 'start': 44, 'end': 49}, {'entity_group': 'loc', 'score': 0.9998049, 'word': 'تهران', 'start': 122, 'end': 127}, {'entity_group': 'loc', 'score': 0.95286226, 'word': 'استونی در حوضه بالتیک', 'start': 237, 'end': 258}, {'entity_group': 'loc', 'score': 0.99962044, 'word': 'ایران', 'start': 322, 'end': 327}, {'entity_group': 'loc', 'score': 0.9822543, 'word': 'افغانستان', 'start': 379, 'end': 388}, {'entity_group': 'loc', 'score': 0.99977845, 'word': 'خراسان', 'start': 421, 'end': 427}, {'entity_group': 'loc', 'score': 0.99959546, 'word': 'المان', 'start': 504, 'end': 509}, {'entity_group': 'loc', 'score': 0.9987937, 'word': 'نايورو', 'start': 690, 'end': 696}, {'entity_group': 'loc', 'score': 0.9996511, 'word': 'اقیانوسیه', 'start': 700, 'end': 709}, {'entity_group': 'org', 'score': 0.9978137, 'word': 'ایرانی', 'start': 730, 'end': 736}, {'entity_group': 'loc', 'score': 0.99966, 'word': 'یوگسلاوی', 'start': 960, 'end': 968}, {

In [ ]:
#pipe = pipeline("token-classification", model="./saved_model_4", tokenizer=tokenizer, aggregation_strategy="simple")

from transformers import pipeline

# assert that we use fast version
assert tokenizer.is_fast

# instantiate pipeline
classifier_1 = pipeline(
    "token-classification",
    model="/content/drive/MyDrive/parsBert/full_finetuned_model",
    tokenizer=tokenizer,
    stride=STRIDE,  # additional parameter
    aggregation_strategy='max'
)

# standard token classification pipeline expects untokenized input
sentence = " ".join(dataset['test']['tokens'][1])
# retokenize sentence for subsequent print statements
tokenized_sentence = tokenizer(
    sentence,
    truncation=True,
    padding="max_length",
    max_length=WINDOW_LENGTH,
    stride=STRIDE,
    return_overflowing_tokens=True,
    is_split_into_words=False
)
# get text length (i.e. number of characters) of first window
text_window_1 = " ".join(
    [tokenizer.decode(tkn) for tkn in tokenized_sentence['input_ids'][0]]
)
n_words = len(dataset['test']['tokens'][1])

print(f"total number of words: {n_words}, total length of text: {len(sentence)}")
# returns: total number of words: 1350, total length of text: 6899
print(f"length of text in first window: {len(text_window_1)}")
# returns: length of text in first window: 2542

output = classifier_1(sentence)

last_token = output[-1]
print(f"last word tagged at text position: {last_token['start']}-{last_token['end']}")
# returns: last word tagged at text position: 6862-6869

total number of words: 37, total length of text: 169
length of text in first window: 3028
last word tagged at text position: 127-136


In [ ]:
print(output)

[{'entity_group': 'pers', 'score': 0.9999149, 'word': 'طاهایی', 'start': 0, 'end': 6}, {'entity_group': 'event', 'score': 0.9877322, 'word': 'هفته دولت', 'start': 127, 'end': 136}]
